## Fruit Neural Network (with KERAS & Tensorflow backend)
Dataset https://www.kaggle.com/moltean/fruits/data

In [1]:
import os
import numpy as np
#from keras.models import Sequential
#from keras.layers import Activation, Dropout, Flatten, Dense
#from keras.preprocessing.image import ImageDataGenerator
#from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D,Conv2D
#from keras import optimizers

import tensorflow as tf
import keras
import keras.backend as K
import keras.layers as KL
import keras.engine as KE
import keras.models as KM
from keras.preprocessing.image import ImageDataGenerator

c:\users\oorra\appdata\local\continuum\anaconda3\envs\maskrcnn\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [74]:
batch_size = 1500
num_classes = 65
epochs = 10

# input image dimensions
img_rows, img_cols = 100, 100

train_data_dir = 'fruits-360/Training'
validation_data_dir = 'fruits-360/Validation'

In [75]:

# used to rescale the pixel values from [0, 255] to [0, 1] interval# used  
datagen = ImageDataGenerator(rescale=1./255)

# automagically retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_cols, img_rows),
        batch_size=512,
        class_mode='categorical')

validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_cols, img_rows),
        batch_size=128,
        class_mode='categorical')



Found 32426 images belonging to 65 classes.
Found 10903 images belonging to 65 classes.


In [76]:
#model = Sequential()
#model.add(Convolution2D(32, 3, 3, input_shape=(img_width, img_height,3)))
#model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))

#model.add(Convolution2D(32, 3, 3))
#model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))

#model.add(Convolution2D(64, 3, 3))
#model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))

#model.add(Flatten())
#model.add(Dense(64))
#model.add(Activation('relu'))
#model.add(Dropout(0.5))
#model.add(Dense(1))
#model.add(Activation('sigmoid'))

model = KM.Sequential()
model.add(KL.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(img_cols, img_rows,3)))
model.add(KL.Conv2D(64, (3, 3), activation='relu'))
model.add(KL.Conv2D(128, (3, 3), activation='relu'))
model.add(KL.MaxPooling2D(pool_size=(2, 2)))
model.add(KL.Dropout(0.25))
model.add(KL.Flatten())
model.add(KL.Dense(128, activation='relu'))
model.add(KL.Dropout(0.5))
model.add(KL.Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [77]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=100,
        #use_multiprocessing=True,
        workers=2)

Epoch 1/10
2/2 [==============================] - 639s 319s/step - loss: 0.1084 - acc: 0.9527 - val_loss: 0.2048 - val_acc: 0.9550
Epoch 2/10
2/2 [==============================] - 660s 330s/step - loss: 0.1121 - acc: 0.9553 - val_loss: 0.2039 - val_acc: 0.9511
Epoch 3/10
2/2 [==============================] - 660s 330s/step - loss: 0.0912 - acc: 0.9613 - val_loss: 0.2017 - val_acc: 0.9536
Epoch 4/10
2/2 [==============================] - 649s 325s/step - loss: 0.0970 - acc: 0.9607 - val_loss: 0.2080 - val_acc: 0.9545
Epoch 5/10
1/2 [==============>...............] - ETA: 38s - loss: 0.0894 - acc: 0.9587

In [9]:
#Score eval: [0.18850819604188968, 0.954144365689559]
score = model.evaluate_generator(validation_generator, steps=batch_size, workers = 1)
print("Score eval: " + str(score))

scores = model.predict_generator(validation_generator, steps=batch_size, workers = 1)
print("Score pred: " + str(score))

Score eval: [0.18850819604188968, 0.954144365689559]
Score pred: [0.18850819604188968, 0.954144365689559]


In [53]:
test_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_cols, img_rows),
        batch_size=25,
        class_mode='categorical')

X = test_generator[0][0]
predictions = model.predict(X)

print(np.argmax(test_generator[0][1], axis = 1))
print(np.argmax(predictions, axis = 1))

Found 10903 images belonging to 65 classes.
[41 27 44 23 10 36  6 62 19 53 49 46 16 54 48 61 20  4 13 34 20 42 59  3
 40]
[41 27 44 23 10 36  6 62 19 53 49 46 16 54 48 61 20  4 13 34 19 42 59  3
 40]


In [45]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
    
# Save the model weights.
weight_path = 'model.h5'
model.save_weights(weight_path)

# Clean up the TF session.
#K.clear_session()

In [ ]:

# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])
score = loaded_model.evaluate(validation_generator, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))